# Prefetching

In [62]:
!python -m pip install --upgrade pip

In [63]:
import os
import time
import codecs

import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager
from threading import Thread


In [64]:
USERNAME = "amelia99"
PASSWORD = os.environ.get('password') 
if PASSWORD is None:
    PASSWORD = input("Please enter your password: ")

In [65]:
location_urls = pd.read_csv("location_urls.csv")
location_urls

,name,url
0,home,https://farmrpg.com/
1,farm,https://farmrpg.com/#!/xfarm.php?id=343807
2,explore,https://farmrpg.com/#!/explore.php
3,mount banon,https://farmrpg.com/#!/area.php?id=8
4,forest,https://farmrpg.com/#!/area.php?id=7
5,help,https://farmrpg.com/#!/quests.php
6,inventory,https://farmrpg.com/#!/inventory.php
7,login,https://farmrpg.com/#!/login.php
8,home,https://farmrpg.com/#!/index.php
9,fish,https://farmrpg.com/#!/fish.php


In [66]:
location_xpaths = pd.read_csv("location_xpaths.csv", delimiter=";")
location_xpaths

,name,xpath
0,farm,"//div[@class=""view view-main navbar-through""]/..."
1,inventory,"//div[@class=""view view-main navbar-through""]/..."
2,explore,"//div[@class=""view view-main navbar-through""]/..."
3,fish,"//div[@class=""view view-main navbar-through""]/..."
4,workshop,"//div[@class=""view view-main navbar-through""]/..."
5,town,"//div[@class=""view view-main navbar-through""]/..."


In [67]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
MAIN_URL = "https://farmrpg.com/"
driver.get(MAIN_URL)

# Helper Functions

In [68]:
def get_current_location():
    # print(driver.current_url)

    try:
        return location_urls[location_urls["url"] == driver.current_url].iloc[0,0]
    except:
        print(f"Location {driver.current_url} not found in locations.csv")
        return None

In [69]:
def is_refresh_button_available():
    return bool(len(driver.find_elements(By.XPATH, "//i[@aria-label='Refresh Game']")))

In [70]:
def goto(target_location):
    target_location = target_location.lower()
    current_location = get_current_location()
    if current_location != target_location:
        print(f"GOTO:  {current_location} -> {target_location}")
        
        driver.get(MAIN_URL)
        
        if target_location == "home":
            return
        elif target_location in location_xpaths["name"].values.tolist():
            xpath =  location_xpaths.loc[location_xpaths["name"] == target_location,"xpath"].iloc[0]
            # print(xpath)
            WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpath)))
            driver.find_element(By.XPATH, xpath).click()
            time.sleep(0.1)
        else:
            print("Sorry, this Location has not been added yet!")

In [71]:
def is_logged_in():
    goto("home")

    if driver.find_element(By.XPATH, "//div[@id='logged_in_username']").get_attribute("innerHTML") == USERNAME:
        return True
    return False

In [72]:
def login_user(username, password):
    if is_logged_in():
        print(f"Already logged in as {username}")
        return

    login_div = driver.find_element(By.XPATH, "//div[contains(text(), 'Login')]")
    login_div.click()

    time.sleep(1)

    username_input = driver.find_element(By.XPATH, "//input[@name='username']")
    username_input.send_keys(username)

    password_input = driver.find_element(By.XPATH, "//input[@name='password']")
    password_input.send_keys(password)

    login_button = driver.find_element(By.XPATH, "//input[@value='Login']")
    login_button.click()

    time.sleep(1)

    if is_logged_in():
        print(f"Successfully logged in as {username}")
    else:
        login_user(username, password)

login_user(USERNAME, PASSWORD)


Location https://farmrpg.com/index.php not found in locations.csv
GOTO:  None -> home
Successfully logged in as amelia99


In [73]:
def set_farm_url():
    goto("home")
    
    farm_id = driver.find_element(By.XPATH, "//a[contains(@href, 'xfarm.php?id')]").get_attribute("href").split("id=")[-1]
    location_urls.loc[location_urls["name"] == "farm","url"] = f"https://farmrpg.com/#!/xfarm.php?id={farm_id}"
    print(f"Set Farm URL to {location_urls.loc[location_urls['name'] == 'farm','url'].iloc[0]}")

set_farm_url()
location_urls
    

Set Farm URL to https://farmrpg.com/#!/xfarm.php?id=343807


,name,url
0,home,https://farmrpg.com/
1,farm,https://farmrpg.com/#!/xfarm.php?id=343807
2,explore,https://farmrpg.com/#!/explore.php
3,mount banon,https://farmrpg.com/#!/area.php?id=8
4,forest,https://farmrpg.com/#!/area.php?id=7
5,help,https://farmrpg.com/#!/quests.php
6,inventory,https://farmrpg.com/#!/inventory.php
7,login,https://farmrpg.com/#!/login.php
8,home,https://farmrpg.com/#!/index.php
9,fish,https://farmrpg.com/#!/fish.php


In [74]:
def go_to_fish_location(target_location=None):
    goto("fish")
        
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//ul//a[contains(@href,'fishing')]//div[@class='item-title']")))
    locations = driver.find_elements(By.XPATH, "//ul//a[contains(@href,'fishing')]//div[@class='item-title']")

    if target_location is None:
        locations[-1].click()
        return
    
    for location in locations:
        if target_location.lower() in location.get_attribute("innerHTML").lower():
            location.click()
            return

    print(f"Location {target_location} not found!")

# Info Functions

In [75]:
def is_harvest_available():
    goto("home")

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'xfarm.php?id')]//span[contains(@class,'ready ready')]")))
    message = driver.find_element(By.XPATH, "//a[contains(@href, 'xfarm.php?id')]//span[contains(@class,'ready ready')]").get_attribute("innerHTML")

    if message == "":
        return False
    elif " Growing" in message:
        return False
    elif " READY!" in message:
        return True

    

In [76]:
def get_available_crops(crop_select=None):
    goto("farm")
    
    if crop_select is None:
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//select[@class='seedid inlineinputlg']")))
        crop_select = Select(driver.find_element(By.XPATH, '//select[@class="seedid inlineinputlg"]'))

    return pd.DataFrame([[o.get_attribute("data-name"), o.get_attribute("data-amt")] for o in crop_select.options], columns=["name", "amount"]).dropna()


In [77]:
def get_inventory():
    goto("inventory")

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//li[@class='list-group-title item-divider']")))
    item_divs = driver.find_elements(By.XPATH, "//li[@class='group-items']/a")

    inventory_items = pd.DataFrame({
        "name": [item.find_element(By.XPATH, ".//div[@class='item-title']/strong").text for item in item_divs],
        "id": [int(item.get_attribute("href").replace("https://farmrpg.com/item.php?id=","")) for item in item_divs],
        "amount": [int(item.find_element(By.XPATH, ".//div[@class='item-after']").text) for item in item_divs],

    })
    return inventory_items


In [78]:
def get_craftable_items():
    goto("workshop")

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'favcraftitems')]")))
    favourite_list = driver.find_elements(By.XPATH, "//div[contains(@class,'favcraftitems')]//li")
    favourite_inputs = driver.find_elements(By.XPATH, "//div[contains(@class,'favcraftitems')]//li//div[@class='item-title']//input")
    
    craftable_items = pd.DataFrame(
        {
            "name": [item.get_attribute("data-name") for item in favourite_list],
            "amount": [int(item.get_attribute("value")) for item in favourite_inputs]
        }
    )

    return craftable_items.sort_values("amount", ascending=False)


In [79]:
def get_fish_locations():
    goto("fish")
        
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//ul//a[contains(@href,'fishing.php')]//div[@class='item-title']")))
    locations = driver.find_elements(By.XPATH, "//ul//a[contains(@href,'fishing.php')]//div[@class='item-title']")
    return [location.get_attribute("innerHTML").split("<br>")[0] for location in locations]


# Action Functions

In [80]:
def harvest():
    goto("farm")
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Harvest All')]")))
    time.sleep(0.01)
    try:
        driver.find_element(By.XPATH, '//a[contains(text(), "Harvest All")]').click()
    except Exception as e:
        driver.refresh()
        driver.find_element(By.XPATH, '//a[contains(text(), "Harvest All")]').click()


In [81]:
def plant_crop(crop_name):
    goto("farm")
    
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//select[@class='seedid inlineinputlg']")))
    crop_select = Select(driver.find_element(By.XPATH, '//select[@class="seedid inlineinputlg"]'))

    available_crops = get_available_crops(crop_select=crop_select)

    crop_name = crop_name[0].upper() + crop_name[1:].lower()
    if "seeds" in crop_name.lower():
        crop_name = crop_name.replace("seeds","").replace("Seeds","").replace(" ","")
    crop_name += " Seeds"


    crop_info = available_crops[available_crops.name == crop_name]

    if crop_info.empty:
        print(f"Could not find crop {crop_name} in available crops")
        # TODO: implement buying
        return

    crop_select.select_by_index(crop_info.index.values[0])

    driver.find_element(By.XPATH, '//a[contains(text(), "Plant All")]').click()
    print(f'Planted {crop_name}')

In [100]:
def buy(item_name, amount=None):
    goto("town")
    item_name = " ".join([item_word[0].upper()+item_word[1:].lower() for item_word in item_name.split(" ")])

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[@href='store.php']")))
    driver.find_element(By.XPATH, "//a[@href='store.php']").click()

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@data-page='store']")))
    buy_button = driver.find_element(By.XPATH, f"//button[@data-name='{item_name}']")

    driver.execute_script("arguments[0].scrollIntoView({ block: 'center', inline: 'center', offsetTop: -200 });", buy_button)
    buy_button.click()
    time.sleep(0.5)

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@class='actions-modal modal-in']")))
    driver.find_element(By.XPATH, "//div[@class='actions-modal-button']").click()

    time.sleep(0.5)
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//span[@class='modal-button modal-button-bold']")))
    driver.find_element(By.XPATH, "//span[@class='modal-button']").click()



In [94]:
def replenish_stamina():
    if driver.current_url != "https://farmrpg.com/#!/fishing.php?id=2":
        go_to_fish_location("farm pond")
    
    time.sleep(0.5)
    fish_divs = driver.find_elements(By.XPATH, "//img[@src='/img/items/fish.png']")

    def fish_coming_up(driver):
        for fish in fish_divs:
            if "catch" in fish.get_attribute("class"):
                return fish
    
    try:
        max_stamina = int(driver.find_element(By.XPATH, "//a[@href='supply.php']").get_attribute("innerHTML").split(" ")[0])
    except:
        print("Stamina is full!")
        return
        
    
    while True:
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href,'changebait')]")))
        if int(driver.find_element(By.XPATH, "//a[contains(@href,'changebait.php')]/../strong").text) == 0:
            print("No bait left!")
            buy("worms")

        catchable = WebDriverWait(driver, 5).until(fish_coming_up)
        ActionChains(driver).move_to_element(catchable.find_element(By.XPATH, '..')).click().perform()

        time.sleep(0.2)

        try:
            WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class,'fishcaught finalcatch')]")))
            driver.find_element(By.XPATH, "//div[contains(@class,'fishcaught finalcatch')]").click()
            time.sleep(0.1)
        except:
            time.sleep(0.1)
            pass
            
        current_stamina = int(driver.find_element(By.XPATH, "//strong[@id='stamina']").get_attribute("innerHTML"))
        if current_stamina == max_stamina:
            break
        

In [91]:
def explore(target_location=None, num=5, rate=10, replenish=True):
    if "area" not in driver.current_url:
        goto("explore")
            
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//ul//a[contains(@href,'area')]//div[@class='item-title']")))
        locations = driver.find_elements(By.XPATH, "//ul//a[contains(@href,'area')]//div[@class='item-title']")
        
        if target_location is None:
            locations[-1].click()
        else:
            for location in locations:
                if target_location.lower() == location.get_attribute("innerHTML").split("<br>")[0].lower():
                    driver.execute_script("arguments[0].scrollIntoView({ block: 'center', inline: 'center', offsetTop: -200 });", location)
                    location.click()
                    break
            else:
                print(f"Could not find location {target_location}")
                return

    time.sleep(0.5)

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@id='exploreconsole']")))
    tab = driver.find_element(By.XPATH, "//div[@id='exploreconsole']")

    for i in range(num):
        if "You have run out of stamina and cannot continue. Eat Apples, Drink OJ or fish at the" in tab.get_attribute("innerHTML"):
            print("No stamina left")
            if replenish: 
                replenish_stamina()
                explore(target_location=target_location, num=num-i, rate=rate, replenish=True)
                break
            else:
                break
        tab.click()
        time.sleep(1/rate)


In [85]:
def fish_manually(target_location=None, num=3):
    
    if "fishing" not in driver.current_url:
        go_to_fish_location(target_location)

    time.sleep(0.5)
    fish_divs = driver.find_elements(By.XPATH, "//img[@src='/img/items/fish.png']")

    def fish_coming_up(driver):
        for fish in fish_divs:
            if "catch" in fish.get_attribute("class"):
                return fish
    
    time.sleep(1)

    for i in range(num):
        catchable = WebDriverWait(driver, 5).until(fish_coming_up)
        ActionChains(driver).move_to_element(catchable.find_element(By.XPATH, '..')).click().perform()

        time.sleep(0.2)

        try:
            WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class,'fishcaught finalcatch')]")))
            driver.find_element(By.XPATH, "//div[contains(@class,'fishcaught finalcatch')]").click()
            time.sleep(0.1)
        except:
            pass
        # ActionChains(driver).move_to_element(driver.find_element(By.XPATH, "//div[contains(@class,'fishcaught')]")).click().perform()


In [86]:
def fish_with_nets(target_location=None, num=5):
    if "fishing" not in driver.current_url:
        go_to_fish_location(target_location)

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'castnet')]")))
    net_div = driver.find_element(By.XPATH, "//div[contains(@class,'castnet')]")

    if int(net_div.find_element(By.XPATH, ".//span[@class='netcount']").text) == 0:
        print("No nets left!")
        return

    for i in range(num):
        net_div.click()
        time.sleep(0.1)



In [87]:
def craft_favourites():
    goto("workshop")

    i = 10

    while True:
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'favcraftitems')]")))
        favourite_craft_buttons = driver.find_elements(
            By.XPATH, 
            "//div[contains(@class,'favcraftitems')]//li//div[@class='item-after']//button[contains(@class,'disable-select button btngreen craftbtnnc craftbtn')]"
        )
        if len(favourite_craft_buttons) == 0:
            break
        
        driver.execute_script("arguments[0].scrollIntoView({ block: 'center', inline: 'center', offsetTop: -200 });", favourite_craft_buttons[0])
        favourite_craft_buttons[0].click()
        time.sleep(1)


In [88]:
def get_sellable_items():
    goto("town")
    time.sleep(0.5)

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[@href='market.php']")))
    driver.find_element(By.XPATH, "//a[@href='market.php']").click()

    time.sleep(0.5)

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[@class='sellallunlockedmaxbtn']")))


    sell_buttons = driver.find_elements(By.XPATH, "//button[contains(@class,'sellbtn')]")
    number_inputs = driver.find_elements(By.XPATH, "//div[@class='page-content']//div[@class='card-content']//ul//div[@class='item-title']//input")
    assert len(sell_buttons) == len(number_inputs), "Number of sell buttons and number inputs do not match!"

    sellable_items = pd.DataFrame({
        "name": [item.get_attribute("data-name") for item in sell_buttons],
        "id": [int(item.get_attribute("data-id")) for item in sell_buttons],
        "price": [int(item.get_attribute("data-price")) for item in number_inputs],
        "amount": [int(item.get_attribute("value")) for item in number_inputs]
    })
    sellable_items["total_price"] = sellable_items["price"] * sellable_items["amount"]
    
    return sellable_items.sort_values("amount", ascending=False)


In [89]:
sellable_items = ["sturdy shield","sturdy sword","fancy pipe"]

def sell_items(items=sellable_items, min_inventory=200, buffer=50):
    sellable_items = get_sellable_items()

    sellable_items["sellable_amount"] = np.maximum(0, sellable_items["amount"].values - min_inventory)
    
    sellable_items.loc[sellable_items["name"].isin(items), "sellable_amount"] = sellable_items.loc[sellable_items["name"].isin(items), "amount"]
    sellable_items = sellable_items[sellable_items["sellable_amount"] > buffer]

    # print(sellable_items)

    for i, row in sellable_items.iterrows():
        print(f"Selling {row['sellable_amount']:>4} {row['name']:<16} for {row['total_price']:>8}")

        driver.find_element(By.XPATH, f"//input[@data-id='{row['id']}']").clear()
        driver.find_element(By.XPATH, f"//input[@data-id='{row['id']}']").send_keys(str(row["sellable_amount"]))
        driver.find_element(By.XPATH, f"//input[@data-id='{row['id']}']").send_keys(Keys.ENTER)
        time.sleep(0.2)

        sell_button = driver.find_element(By.XPATH, f"//button[@data-id='{row['id']}' and contains(@class, 'sellbtn')]")
        driver.execute_script("arguments[0].scrollIntoView({ block: 'center', inline: 'center', offsetTop: -200 });", sell_button)
        sell_button.click()
        time.sleep(0.2)

        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@class='actions-modal modal-in']")))
        driver.find_element(By.XPATH, "//div[@class='actions-modal-button']").click()

        time.sleep(0.2)
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//span[@class='modal-button modal-button-bold']")))
        driver.find_element(By.XPATH, "//span[@class='modal-button modal-button-bold']").click()

        time.sleep(0.5)


# Loops

In [102]:

while True:
    try:
        harvest()
        buy("hops seeds")
        plant_crop("hops")
        
        # explore(target_location="black rock canyon", num=50)
        explore(target_location=None, num=30)
        explore(target_location="forest", num=20)
        craft_favourites()
        sell_items()

        # check inv and do something (FN, townsfolk, specials, help requests)
        inv = get_inventory()
        if inv.loc[inv["name"] == "Fishing Net", "amount"].iloc[0] > 300:
            fish_with_nets(num=200)

    except Exception as e:
        print("--------------------------------------------------\n", e)
        time.sleep(1)
        pass


GOTO:  black rock canyon -> farm
GOTO:  farm -> town
Location https://farmrpg.com/#!/store.php not found in locations.csv
GOTO:  None -> farm
Planted Hops Seeds
GOTO:  farm -> explore
No stamina left
GOTO:  black rock canyon -> fish
GOTO:  farm pond -> explore


In [ ]:
while True:
    try:
        explore(target_location="black rock canyon", num=100)
    except Exception as e:
        print("--------------------------------------------------\n",e)
        pass


In [ ]:
# try threads and tab windows